In [16]:
import geopandas as gpd
import pandas as pd

# That didn't load enough data, the query limit is 65000, so let's try a different approach
# Base URL for the Parcel Points feature layer in GeoJSON format
base_url = "https://geodata.md.gov/imap/rest/services/PlanningCadastre/MD_PropertyData/MapServer/0/query"
max_records = 10000  # Maximum number of records per query


def fetch_data(base_url, max_records):
    all_data = gpd.GeoDataFrame()
    offset = 0

    while True:
        query_params = {
            "where": "JURSCODE%3D'BACI'%20AND%20CONVEY1%3D1",
            "outFields": "JURSCODE,ACCTID,DIGXCORD,DIGYCORD,GEOGCODE,ZIPCODE,MAP,ZONING,ZNCHGDAT,RZREALDAT,LU,DESCLU,ACRES,LANDAREA,LUOM,YEARBLT,LASTINSP,LASTASSD,ASSESSOR,TRANSNO1,CONVEY1,TRADATE,CONSIDR1,MORTGAG1,NFMLNDVL,NFMIMPVL,PLNDEVDAT,NPRCTSTDAT,NPRCAREA,NPRCLUOM,HOMQLDAT,BLDG_STORY,BLDG_UNITS,APRTMENT,TRAILER,SPECIAL,OTHER,PTYPE,SDATWEBADR,EXISTING,MDPVDATE,HOMQLCOD,RESIDENT,NFMTTLVL,SDATDATE",
            "outSR": "4326",
            "f": "geojson",
            "resultOffset": offset,
            "resultRecordCount": max_records,
        }
        query_url = f"{base_url}?{'&'.join([f'{k}={v}' for k, v in query_params.items()])}"

        data_chunk = gpd.read_file(query_url)

        if data_chunk.empty:
            break

        all_data = pd.concat([all_data, data_chunk], ignore_index=True)
        offset += max_records

    return all_data


gdf = fetch_data(base_url, max_records)
gdf.head()

,JURSCODE,ACCTID,DIGXCORD,DIGYCORD,GEOGCODE,ZIPCODE,MAP,ZONING,ZNCHGDAT,RZREALDAT,...,OTHER,PTYPE,SDATWEBADR,EXISTING,MDPVDATE,HOMQLCOD,RESIDENT,NFMTTLVL,SDATDATE,geometry
0,BACI,0301011738 001,435607.3,180454.0,82,21231,0001,R-8,20220806,,...,0,2,https://sdat.dat.maryland.gov/RealProperty/Pag...,UPDT2017_18,2022SEP,A,1,751900,2023MAY,POINT (-76.58725 39.29157)
1,BACI,0301011738 002,435612.6,180454.7,82,21231,0001,R-8,,,...,0,2,https://sdat.dat.maryland.gov/RealProperty/Pag...,UPDT2017_18,2022SEP,A,1,553600,2023MAY,POINT (-76.58719 39.29157)
2,BACI,0301011738 003,435617.9,180455.0,82,21231,0001,R-8,20220802,,...,0,2,https://sdat.dat.maryland.gov/RealProperty/Pag...,UPDT2017_18,2022SEP,,1,478300,2023MAY,POINT (-76.58712 39.29158)
3,BACI,0301011738 004,435622.4,180454.8,82,21231,0001,R-8,,,...,0,2,https://sdat.dat.maryland.gov/RealProperty/Pag...,UPDT2017_18,2022SEP,A,1,558100,2023MAY,POINT (-76.58707 39.29157)
4,BACI,0301011738 005,435628.5,180455.1,82,21231,0001,R-8,,,...,0,2,https://sdat.dat.maryland.gov/RealProperty/Pag...,UPDT2017_18,2022SEP,A,1,693000,2023MAY,POINT (-76.58700 39.29158)


In [18]:
len(gdf)

96960

In [19]:
gdf["assessed_difference"] = gdf["NFMTTLVL"] - gdf["CONSIDR1"]

In [20]:
gdf["TRADATE"].head()

0    20160719
1    20170426
2    20220721
3    19950725
4    20130826
Name: TRADATE, dtype: object

In [21]:
# check for missing values
gdf["TRADATE"].isnull().sum()

0

In [22]:
gdf["TRADATE"].str.len().value_counts()

TRADATE
8    96959
1        1
Name: count, dtype: int64

In [23]:
# investigate why there is a date with 1 character
gdf[gdf["TRADATE"].str.len() == 1].T

,10376
JURSCODE,BACI
ACCTID,0307141622 012
DIGXCORD,435759.2
DIGYCORD,181342.8
GEOGCODE,81
ZIPCODE,21205
MAP,0007
ZONING,R-8
ZNCHGDAT,
RZREALDAT,


In [24]:
# drop date with 1 character
gdf = gdf[gdf["TRADATE"].str.len() != 1]
# current format is YYYYMMDD
gdf["sale_date"] = pd.to_datetime(gdf["TRADATE"], format="%Y%m%d")
gdf["sale_date"].head()

0   2016-07-19
1   2017-04-26
2   2022-07-21
3   1995-07-25
4   2013-08-26
Name: sale_date, dtype: datetime64[ns]

In [25]:
gdf["assessment_year"] = gdf["LASTASSD"].astype(str).str[:4].astype(int)
gdf["assessment_year"].head()

0    2021
1    2021
2    2021
3    2021
4    2021
Name: assessment_year, dtype: int64

In [26]:
gdf["sold_after_assessment"] = (
    gdf["sale_date"].dt.year > gdf["assessment_year"]
)

In [27]:
# Dropw rows where sale price is 0
gdf = gdf[gdf["CONSIDR1"] > 0]
# Keep only data from after 2020
gdf = gdf[gdf["sale_date"].dt.year >= 2020]

In [31]:
# Create a folium heatmap of the assessed difference for properties
import folium
from folium.plugins import HeatMap

under_assessed = gdf[(gdf["assessed_difference"] < 0)]

# Create a base map
m = folium.Map(
    location=[gdf.geometry.y.mean(), gdf.geometry.x.mean()],
    tiles="cartodbpositron",
    zoom_start=11,
)

# Heatmap of the assessed difference
heat_data = [
    [row["geometry"].y, row["geometry"].x, row["assessed_difference"]]
    for index, row in under_assessed.iterrows()
]
HeatMap(heat_data, radius=8, max_zoom=13).add_to(m)

m

In [29]:
# do the same for over assessed properties
over_assessed = gdf[(gdf["assessed_difference"] > 0)]

# Create a base map
m = folium.Map(
    location=[gdf.geometry.y.mean(), gdf.geometry.x.mean()],
    tiles="cartodbpositron",
    zoom_start=11,
)

# Heatmap of the assessed difference
heat_data = [
    [row["geometry"].y, row["geometry"].x, row["assessed_difference"]]
    for index, row in over_assessed.iterrows()
]
HeatMap(heat_data, radius=8, max_zoom=13).add_to(m)
m

In [33]:
# Filter to get under-assessed properties
under_assessed = gdf[gdf["assessed_difference"] < 0]

# Create a base map centered around the mean coordinates of the data
m = folium.Map(
    location=[gdf.geometry.y.mean(), gdf.geometry.x.mean()],
    tiles="cartodbpositron",
    zoom_start=11,
)

# Prepare data for the heatmap
# Each point consists of latitude, longitude, and a weight (assessed difference)
# Corrected Heatmap Data Preparation
heat_data = [
    [point.geometry.y, point.geometry.x, -point.assessed_difference]
    for point in under_assessed.itertuples()
]


# Create and add the heatmap to the map
# The weight is given by the negative assessed difference
HeatMap(heat_data, radius=8, max_zoom=13).add_to(m)

# Display the map
m